In [2]:
import twitter

CONSUMER_KEY = ''
CONSUMER_SECRET = ''
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print twitter_api

In [3]:
import json

In [4]:
q = '#feelthebern'
# Search in a 12 mile radius of the center of Milwaukee County, WI
geocode = '43.016,-87.884,12mi'
results = twitter_api.search.tweets(q = q, geocode = geocode, count = 100)
statuses = results['statuses']
print(len(statuses))

100


In [5]:
for _ in range(100):
    print "Length of statuses", len(statuses)
    try:
        next_results = results['search_metadata']['next_results']
    except KeyError, e: # No more results when next_results doesn't exist
        break
        
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    results = twitter_api.search.tweets(q = q, geocode = geocode, count = '100', max_id = kwargs['max_id'])
    statuses += results['statuses']

Length of statuses 100
Length of statuses 117
